### Dependencies and setup 

In [54]:
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

### World happiness data cleaning

In [55]:
world_happiness = "./Resources/World_Happiness_Data/2015.csv"
world_happiness_df = pd.read_csv(world_happiness)
world_happiness_df.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [3]:
# Create a filtered dataframe from specific columns
happiness_2015 = ["Country", "Happiness Rank", "Happiness Score"]
happiness_transformed= world_happiness_df[happiness_2015].copy()

# Rename the column headers
happiness_transformed = happiness_transformed.rename(columns={"Country": "country",
                                                          "Happiness Rank": "happiness_rank",
                                                          "Happiness Score": "happiness_score"})

happiness_transformed.head()

,country,happiness_rank,happiness_score
0,Switzerland,1,7.587
1,Iceland,2,7.561
2,Denmark,3,7.527
3,Norway,4,7.522
4,Canada,5,7.427


### World bank data cleaning

In [4]:
# Study data files
country_population = "./Resources/World_Bank_Data/country_population.csv"
fertility_rate = "./Resources/World_Bank_Data/fertility_rate.csv"
life_expectancy = "./Resources/World_Bank_Data/life_expectancy.csv"

### Fertility rate

In [5]:
f_rate = pd.read_csv(fertility_rate)
f_rate.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,4.820,4.655,4.471,4.271,4.059,3.842,...,1.763,1.764,1.769,1.776,1.783,1.791,1.796,1.800,1.801,1.800
1,Afghanistan,AFG,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,7.450,7.450,7.450,7.450,7.450,7.450,...,6.460,6.254,6.038,5.816,5.595,5.380,5.174,4.981,4.802,4.635
2,Angola,AGO,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,7.478,7.524,7.563,7.592,7.611,7.619,...,6.368,6.307,6.238,6.162,6.082,6.000,5.920,5.841,5.766,5.694
3,Albania,ALB,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,6.489,6.401,6.282,6.133,5.960,5.773,...,1.668,1.650,1.646,1.653,1.668,1.685,1.700,1.710,1.714,1.713
4,Andorra,AND,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.180,1.250,1.190,1.270,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# clean data
f_rate_df = f_rate.loc[:, ["Country Name","Country Code", "2015"]]
f_rate_df
f_rate_df.rename(columns = {'2015':'fertility_rate_2015', 'Country Code': 'country_code'}, inplace = True) 
f_rate_df

,Country Name,country_code,fertility_rate_2015
0,Aruba,ABW,1.801
1,Afghanistan,AFG,4.802
2,Angola,AGO,5.766
3,Albania,ALB,1.714
4,Andorra,AND,NaN
...,...,...,...
259,Kosovo,XKX,2.090
260,"Yemen, Rep.",YEM,4.104
261,South Africa,ZAF,2.485
262,Zambia,ZMB,5.041


f_rate = pd.read_csv(fertility_rate)
f_rate.head()

### Life expectency

In [7]:
l_expectancy = pd.read_csv(life_expectancy)
l_expectancy.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.662,66.074,66.444,66.787,67.113,67.435,...,74.576,74.725,74.872,75.016,75.158,75.299,75.440,75.582,75.725,75.867
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.292,32.742,33.185,33.624,34.060,34.495,...,59.694,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,33.251,33.573,33.914,34.272,34.645,35.031,...,55.096,56.189,57.231,58.192,59.042,59.770,60.373,60.858,61.241,61.547
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,62.279,63.298,64.187,64.911,65.461,65.848,...,75.656,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# clean data
l_expectancy_df = l_expectancy.loc[:, ["Country Name", "2015"]]
l_expectancy_df
l_expectancy_df.rename(columns = {'2015':'life_expectancy_2015'}, inplace = True) 
l_expectancy_df

,Country Name,life_expectancy_2015
0,Aruba,75.725000
1,Afghanistan,63.288000
2,Angola,61.241000
3,Albania,78.174000
4,Andorra,NaN
...,...,...
259,Kosovo,71.346341
260,"Yemen, Rep.",64.743000
261,South Africa,61.981000
262,Zambia,61.397000


### Population

In [9]:
c_population = pd.read_csv(country_population)
c_population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0


In [10]:
# clean data
c_population_df = c_population.loc[:, ["Country Name", "2015"]]
c_population_df
c_population_df.rename(columns = {'2015':'сountry_population_2015'}, inplace = True) 
c_population_df

,Country Name,сountry_population_2015
0,Aruba,104341.0
1,Afghanistan,33736494.0
2,Angola,27859305.0
3,Albania,2880703.0
4,Andorra,78014.0
...,...,...
259,Kosovo,1801800.0
260,"Yemen, Rep.",26916207.0
261,South Africa,55291225.0
262,Zambia,16100587.0


### Merged dataframes

In [11]:
# Merge two dataframes using an outer join
fertility_rate_life_expectancy = pd.merge(f_rate_df, l_expectancy_df, on="Country Name", how="outer")
fertility_rate_life_expectancy

,Country Name,country_code,fertility_rate_2015,life_expectancy_2015
0,Aruba,ABW,1.801,75.725000
1,Afghanistan,AFG,4.802,63.288000
2,Angola,AGO,5.766,61.241000
3,Albania,ALB,1.714,78.174000
4,Andorra,AND,NaN,NaN
...,...,...,...,...
259,Kosovo,XKX,2.090,71.346341
260,"Yemen, Rep.",YEM,4.104,64.743000
261,South Africa,ZAF,2.485,61.981000
262,Zambia,ZMB,5.041,61.397000


In [12]:
# Merge two dataframes using an outer join
merge_data = pd.merge(fertility_rate_life_expectancy, c_population_df, on="Country Name", how="outer")
merge_data
merge_data.rename(columns = {'Country Name':'country_name'}, inplace = True) 
merge_data

,country_name,country_code,fertility_rate_2015,life_expectancy_2015,сountry_population_2015
0,Aruba,ABW,1.801,75.725000,104341.0
1,Afghanistan,AFG,4.802,63.288000,33736494.0
2,Angola,AGO,5.766,61.241000,27859305.0
3,Albania,ALB,1.714,78.174000,2880703.0
4,Andorra,AND,NaN,NaN,78014.0
...,...,...,...,...,...
259,Kosovo,XKX,2.090,71.346341,1801800.0
260,"Yemen, Rep.",YEM,4.104,64.743000,26916207.0
261,South Africa,ZAF,2.485,61.981000,55291225.0
262,Zambia,ZMB,5.041,61.397000,16100587.0


### Country code

In [13]:
country_code_df = merge_data.loc[:, ["country_name","country_code"]]
country_code_df

,country_name,country_code
0,Aruba,ABW
1,Afghanistan,AFG
2,Angola,AGO
3,Albania,ALB
4,Andorra,AND
...,...,...
259,Kosovo,XKX
260,"Yemen, Rep.",YEM
261,South Africa,ZAF
262,Zambia,ZMB


### World GDP

In [14]:
# Import GDP
gdp_file = "Resources/World_GDP/gdp_per_capita.csv"
gdp_df = pd.read_csv(gdp_file)
gdp_df.head()

,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2015,Estimates Start After
0,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",599.99,2013.0
1,Albania,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",3995.38,2010.0
2,Algeria,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",4318.14,2014.0
3,Angola,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",4100.32,2014.0
4,Antigua and Barbuda,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",14414.30,2011.0


In [15]:
# Clean GDP
gdp_cols = ['Country', '2015']
gdp_trans = gdp_df[gdp_cols].copy()
gdp_trans = gdp_trans.rename(columns={"Country": "country",
                                      "2015": 'gdp'})

gdp_trans.head()

,country,gdp
0,Afghanistan,599.99
1,Albania,3995.38
2,Algeria,4318.14
3,Angola,4100.32
4,Antigua and Barbuda,14414.30


In [16]:
world_happiness = "./Resources/World_Happiness_Data/2015.csv"
world_happiness_df = pd.read_csv(world_happiness)
world_happiness_df.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [17]:
# Find just the happiness dataset countries to compare to GDP
wh_country = ["Country"]
wh_trans_df = world_happiness_df[wh_country].copy()
result_hap = wh_trans_df.sort_values(['Country'])
result_hap = result_hap.rename(columns={"Country": 'country'})
result_hap = result_hap.reset_index()
del result_hap['index']
result_hap.head()

,country
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Argentina


In [18]:
gdp_count = ["country"]
result_gdp_df = gdp_trans[gdp_count].copy()
result_gdp_df.head()

,country
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Antigua and Barbuda


In [19]:
# Finding unique countries between the two dataframes
merge_df = pd.concat([result_hap, result_gdp_df]).drop_duplicates(keep=False)
merge_df.head()

,country
31,Congo (Brazzaville)
32,Congo (Kinshasa)
56,Hong Kong
61,Iran
66,Ivory Coast


In [20]:
merge_df.describe()

,country
count,59
unique,59
top,Guyana
freq,1


In [21]:
# Create a filtered dataframe from specific columns
happiness_2015 = ["Country", "Happiness Rank"]
happiness_transformed= world_happiness_df[happiness_2015].copy()

# Rename the column headers
happiness_transformed = happiness_transformed.rename(columns={"Country": "country",
                                                          "Happiness Rank": "happiness_rank",
                                                          })

happiness_transformed.head()

,country,happiness_rank
0,Switzerland,1
1,Iceland,2
2,Denmark,3
3,Norway,4
4,Canada,5


In [22]:
happiness_transformed.describe()

,happiness_rank
count,158.000000
mean,79.493671
std,45.754363
min,1.000000
25%,40.250000
50%,79.500000
75%,118.750000
max,158.000000


In [23]:
merged_df = pd.merge(happiness_transformed, gdp_trans, on="country")
merged_df.head()

,country,happiness_rank,gdp
0,Switzerland,1,80675.31
1,Iceland,2,50854.58
2,Denmark,3,52114.17
3,Norway,4,74822.11
4,Canada,5,43331.96


In [24]:
# Check for any null values in GDP
check = merged_df['gdp'].isnull()
check_df = merged_df[check]
check_df

,country,happiness_rank,gdp
64,Kosovo,69,NaN
141,Syria,156,NaN


In [25]:
cleaned_gdp_df = merged_df.dropna()
cleaned_gdp_df = cleaned_gdp_df.reset_index(drop=True) 
cleaned_gdp_df.head()

,country,happiness_rank,gdp
0,Switzerland,1,80675.31
1,Iceland,2,50854.58
2,Denmark,3,52114.17
3,Norway,4,74822.11
4,Canada,5,43331.96


In [26]:
cleaned_gdp_df.describe()

,happiness_rank,gdp
count,142.000000,142.000000
mean,78.021127,13691.567676
std,46.544186,18866.582650
min,1.000000,305.780000
25%,36.250000,1491.812500
50%,78.500000,5740.855000
75%,118.750000,16879.737500
max,158.000000,101994.090000


In [27]:
original_order = cleaned_gdp_df.reset_index()
original_order['index'] = original_order['index']+1
#og_order = og_order.set_index('index')
#original_order = original_order[['index']]
del original_order['happiness_rank']
original_order= original_order.rename(columns={'index': 'happiness_rank'})
original_order = original_order[['country', 'happiness_rank', 'gdp']]
original_order

,country,happiness_rank,gdp
0,Switzerland,1,80675.31
1,Iceland,2,50854.58
2,Denmark,3,52114.17
3,Norway,4,74822.11
4,Canada,5,43331.96
...,...,...,...
137,Afghanistan,138,599.99
138,Rwanda,139,731.51
139,Benin,140,780.06
140,Burundi,141,305.78


In [28]:
original_happiness_rank = original_order[['happiness_rank']]
original_happiness_rank

,happiness_rank
0,1
1,2
2,3
3,4
4,5
...,...
137,138
138,139
139,140
140,141


In [29]:
cleaned_gdp_df.dtypes

country            object
happiness_rank      int64
gdp               float64
dtype: object

In [30]:
# Sort by GDP
#gdp_sort = cleaned_gdp_df.sort_values(['gdp'], ascending=False)
gdp_sort = original_order.sort_values(['gdp'], ascending=False)
gdp_sort = gdp_sort.reset_index()
del gdp_sort['index']
gdp_sort
#gdp_sort.dtypes

,country,happiness_rank,gdp
0,Luxembourg,17,101994.09
1,Switzerland,1,80675.31
2,Qatar,28,76576.08
3,Norway,4,74822.11
4,United States,15,55805.20
...,...,...,...
137,Niger,130,405.21
138,Madagascar,133,401.77
139,Malawi,118,354.28
140,Central African Republic,134,334.87


In [31]:
# Add gdp_rank
gdp_rank = gdp_sort
gdp_rank['gdp_rank'] = gdp_rank.index +1
gdp_rank

,country,happiness_rank,gdp,gdp_rank
0,Luxembourg,17,101994.09,1
1,Switzerland,1,80675.31,2
2,Qatar,28,76576.08,3
3,Norway,4,74822.11,4
4,United States,15,55805.20,5
...,...,...,...,...
137,Niger,130,405.21,138
138,Madagascar,133,401.77,139
139,Malawi,118,354.28,140
140,Central African Republic,134,334.87,141


In [32]:
#change gdp to currency
gdp_mon = gdp_rank
gdp_mon['gdp'] = gdp_mon[['gdp']].applymap('${:,.2f}'.format)
gdp_mon

,country,happiness_rank,gdp,gdp_rank
0,Luxembourg,17,"$101,994.09",1
1,Switzerland,1,"$80,675.31",2
2,Qatar,28,"$76,576.08",3
3,Norway,4,"$74,822.11",4
4,United States,15,"$55,805.20",5
...,...,...,...,...
137,Niger,130,$405.21,138
138,Madagascar,133,$401.77,139
139,Malawi,118,$354.28,140
140,Central African Republic,134,$334.87,141


### Export all csv files

In [33]:
#Export to final CSV
from IPython.display import FileLink, FileLinks
merge_data.to_csv("./Export/WorldBankData.csv", index=False)
gdp_mon.to_csv("./Export/GDPmon.csv", index=False)
happiness_transformed.to_csv("./Export/HappinessTransformed.csv", index=False)
country_code_df.to_csv("./Export/CountryCode.csv", index=False)

### Connect to local database

In [38]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [42]:
engine.table_names()

['world_happiness',
 'country_code',
 'gdp_happiness',
 'pop_fertility_rate_life_exp',
 'happiness_transformed']

### Use pandas to load csv converted DataFrame into database

In [43]:
happiness_transformed.to_sql(name='happiness_transformed', con=engine, if_exists='append', index=False)

In [44]:
merge_data.to_sql(name='merge_data', con=engine, if_exists='append', index=False)

In [45]:
gdp_mon.to_sql(name='gdp_mon', con=engine, if_exists='append', index=False)

In [46]:
country_code_df.to_sql(name='country_code_df', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the tables

In [47]:
pd.read_sql_query('select * from happiness_transformed', con=engine).head()

,country,happiness_rank
0,Switzerland,1
1,Iceland,2
2,Denmark,3
3,Norway,4
4,Canada,5


In [51]:
pd.read_sql_query('select * from merge_data ', con=engine).head()

,country_name,country_code,fertility_rate_2015,life_expectancy_2015,сountry_population_2015
0,Aruba,ABW,1.801,75.725,104341.0
1,Afghanistan,AFG,4.802,63.288,33736494.0
2,Angola,AGO,5.766,61.241,27859305.0
3,Albania,ALB,1.714,78.174,2880703.0
4,Andorra,AND,NaN,NaN,78014.0


In [49]:
pd.read_sql_query('select * from gdp_mon ', con=engine).head()

,country,happiness_rank,gdp,gdp_rank
0,Luxembourg,17,"$101,994.09",1
1,Switzerland,1,"$80,675.31",2
2,Qatar,28,"$76,576.08",3
3,Norway,4,"$74,822.11",4
4,United States,15,"$55,805.20",5


In [50]:
pd.read_sql_query('select * from country_code_df ', con=engine).head()

,country_name,country_code
0,Aruba,ABW
1,Afghanistan,AFG
2,Angola,AGO
3,Albania,ALB
4,Andorra,AND
